In [69]:
import pandas as pd
import numpy as np

import plotly.express as px
import matplotlib.pyplot as plt
import plotly.subplots as sp
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [70]:
df_features = pd.read_csv('/content/drive/MyDrive/Datasets/walmart/features.csv')
df_stores   = pd.read_csv('/content/drive/MyDrive/Datasets/walmart/stores.csv')
df_train    = pd.read_csv('/content/drive/MyDrive/Datasets/walmart/train.csv')
df_test     = pd.read_csv('/content/drive/MyDrive/Datasets/walmart/test.csv')

In [71]:
df_features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [72]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


In [77]:
def show_null_report(df_features):
  null_values_percent = df_features.isnull().sum()/len(df_features) * 100
  null_report  = pd.DataFrame({'column_name':df_features.isnull().sum(),
                              'null_values_percent':null_values_percent})
  return null_report

In [80]:
show_null_report(df_train)

,column_name,null_values_percent
Store,0,0.0
Dept,0,0.0
Date,0,0.0
Weekly_Sales,0,0.0
IsHoliday,0,0.0


In [81]:
show_null_report(df_test)

,column_name,null_values_percent
Store,0,0.0
Dept,0,0.0
Date,0,0.0
IsHoliday,0,0.0


In [82]:
show_null_report(df_stores)

,column_name,null_values_percent
Store,0,0.0
Type,0,0.0
Size,0,0.0


In [83]:
show_null_report(df_features)

,column_name,null_values_percent
Store,0,0.000000
Date,0,0.000000
Temperature,0,0.000000
Fuel_Price,0,0.000000
MarkDown1,4158,50.769231
MarkDown2,5269,64.334554
MarkDown3,4577,55.885226
MarkDown4,4726,57.704518
MarkDown5,4140,50.549451
CPI,585,7.142857




---


# **Handling missing values**

Not dropping markdowns though they have more than ***50% of the values*** missing due to data loss.

Also as mentioned on the [walmart-recruiting-store-sales-forecasting](https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/data) it is only available after Nov 2011, and is not available for all stores all the time.

Any missing value is marked with an NA so assuming them as ***zeroes***
as there is less data for imputing by mean or median.


In [63]:
# Zeroes for NA values ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
df_features['MarkDown1'] = df_features['MarkDown1'].fillna(0)
df_features['MarkDown2'] = df_features['MarkDown2'].fillna(0)
df_features['MarkDown3'] = df_features['MarkDown3'].fillna(0)
df_features['MarkDown4'] = df_features['MarkDown4'].fillna(0)
df_features['MarkDown5'] = df_features['MarkDown5'].fillna(0)


df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     8190 non-null   float64
 5   MarkDown2     8190 non-null   float64
 6   MarkDown3     8190 non-null   float64
 7   MarkDown4     8190 non-null   float64
 8   MarkDown5     8190 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


Handling missing values of ***CPI(Consumer Price Index)*** and ***Unemployment***

In [64]:

# Create subplots with 1 row and 2 columns
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=['No Outliers', 'Found Outliers'])

# Add boxplots for feature1 and feature2
fig.add_trace(go.Box(y=df_features['CPI'], name='CPI', marker_color = '#0071ce'), row=1, col=1)
fig.add_trace(go.Box(y=df_features['Unemployment'], name='Unemployment', marker_color = '#ffc220'), row=1, col=2)

# Update layout
fig.update_layout(
    title_text='Boxplots for Features',
    plot_bgcolor='rgba(0, 0, 0, 0)'  # Set the background color (transparent in this case) with 0% opacity
)

# Show the plot
fig.show()

Imputating missing values : ***Mean*** or ***Median*** ?

**CPI** :

No outliers in the CPI feature.

As mean defines the ***centeral tendency of the distribution***.

So replacing nan values with **Mean**.

**Unemployment** :

Outliers found in the Unemployment feature.

Mean is affected by outliers.

Median defines the ***centeral tendency of the distribution*** here.

So replacing nan values with **Median**.

In [65]:
df_features['CPI'].fillna(df_features['CPI'].mean(), inplace = True)
df_features['Unemployment'].fillna(df_features['Unemployment'].median(), inplace = True)

In [67]:
show_null_report(df_features)

,column_name,null_values_percent
Store,0,0.0
Date,0,0.0
Temperature,0,0.0
Fuel_Price,0,0.0
MarkDown1,0,0.0
MarkDown2,0,0.0
MarkDown3,0,0.0
MarkDown4,0,0.0
MarkDown5,0,0.0
CPI,0,0.0


***All the missing values handled.***

---



In [87]:
df_train.describe()

,Store,Dept,Weekly_Sales
count,421570.000000,421570.000000,421570.000000
mean,22.200546,44.260317,15981.258123
std,12.785297,30.492054,22711.183519
min,1.000000,1.000000,-4988.940000
25%,11.000000,18.000000,2079.650000
50%,22.000000,37.000000,7612.030000
75%,33.000000,74.000000,20205.852500
max,45.000000,99.000000,693099.360000


***Negative Sales*** do not make sense .

Taking only ***Non Negative Sales*** into consideration .

In [91]:
df_train = df_train[df_train['Weekly_Sales']>=0]
df_train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 420285 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         420285 non-null  int64  
 1   Dept          420285 non-null  int64  
 2   Date          420285 non-null  object 
 3   Weekly_Sales  420285 non-null  float64
 4   IsHoliday     420285 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 16.4+ MB


train set size : ***420285*** samples

In [8]:
df_stores.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [9]:
df_train.info()
df_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB


,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


train set size : ***421570 samples***

# **Exploratory Data Analysis : exploring Sales**

In [92]:
df_test.info()
df_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115064 entries, 0 to 115063
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Store      115064 non-null  int64 
 1   Dept       115064 non-null  int64 
 2   Date       115064 non-null  object
 3   IsHoliday  115064 non-null  bool  
dtypes: bool(1), int64(2), object(1)
memory usage: 2.7+ MB


,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False


test set size : ***115064 samples***

In [96]:
df_train.dtypes

Store             int64
Dept              int64
Date             object
Weekly_Sales    float64
IsHoliday          bool
dtype: object

In [97]:
df_features.dtypes

Store             int64
Date             object
Temperature     float64
Fuel_Price      float64
MarkDown1       float64
MarkDown2       float64
MarkDown3       float64
MarkDown4       float64
MarkDown5       float64
CPI             float64
Unemployment    float64
IsHoliday          bool
dtype: object

In [98]:
# 'Date' found in df_features, df_train, df_test.
# Converting 'Date' field to date_time dtype:
df_features['Date'] = pd.to_datetime(df_features['Date'])
df_train['Date'] = pd.to_datetime(df_train['Date'])
df_test['Date'] = pd.to_datetime(df_test['Date'])

In [99]:
df_train['day'] = df_train['Date'].dt.day
df_train['week'] = df_train['Date'].dt.week
df_train['month'] = df_train['Date'].dt.month
df_train['year'] = df_train['Date'].dt.year
df_train['day_period'] = (df_train['Date'].dt.date-df_train['Date'].dt.date.min()).apply(lambda x:x.days)

<ipython-input-99-51fa6493ca03>:2: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.



In [100]:
df_train

,Store,Dept,Date,Weekly_Sales,IsHoliday,day,week,month,year,day_period
0,1,1,2010-02-05,24924.50,False,5,5,2,2010,0
1,1,1,2010-02-12,46039.49,True,12,6,2,2010,7
2,1,1,2010-02-19,41595.55,False,19,7,2,2010,14
3,1,1,2010-02-26,19403.54,False,26,8,2,2010,21
4,1,1,2010-03-05,21827.90,False,5,9,3,2010,28
...,...,...,...,...,...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False,28,39,9,2012,966
421566,45,98,2012-10-05,628.10,False,5,40,10,2012,973
421567,45,98,2012-10-12,1061.02,False,12,41,10,2012,980
421568,45,98,2012-10-19,760.01,False,19,42,10,2012,987


,Store,Date,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday,Type,Size
0,1,2010-02-05,42.31,2.572,211.096358,8.106,False,A,151315
1,1,2010-02-12,38.51,2.548,211.242170,8.106,True,A,151315
2,1,2010-02-19,39.93,2.514,211.289143,8.106,False,A,151315
3,1,2010-02-26,46.63,2.561,211.319643,8.106,False,A,151315
4,1,2010-03-05,46.50,2.625,211.350143,8.106,False,A,151315
